### Описание

Считываем базу данных приложения по контролю за доходами/расходами и формируем из нее csv-файл для загрузки в новое приложение.

In [38]:
import sqlite3
import numpy as np
import pandas as pd

In [2]:
con = sqlite3.connect('expense_manager')

In [3]:
sql_query = (f"SELECT * FROM  'xpncetb' ")
expence = pd.read_sql(sql_query, con, index_col='id')
expence

,cid,desc,itmdate,amnt,time,img,paymthrd,withwhome,voice,location
id,,,,,,,,,,
1,6,,2014-07-23,300,23:53,,0,None,None,None
2,11,,2014-07-23,200,23:55,,0,None,None,None
3,3,Пущино - Серпухов,2014-07-24,62,14:44,,0,None,None,None
4,7,шампунь\nгель для душа\nдве пасты\nдве щетки,2014-07-24,660,16:4,,0,None,None,None
5,2,Pazzo,2014-07-24,1000,17:14,,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...
2916,6,крупы,2020-03-11,530,16:56,,0,None,None,None
2917,3,москва,2020-03-16,590,16:56,,0,None,None,None
2918,3,москва,2020-03-13,590,16:56,,0,None,None,None


In [4]:
"""
Создаем словарь категорий трат для последующего использования в создании единой таблицы трат
"""

sql_query = (f"SELECT * FROM  'xpncecattb' ")
expence = pd.read_sql(sql_query, con)
expence_category = dict(zip(expence.id, expence.cname))

In [5]:
"""
Создаем словарь категорий приходов для последующего использования в создании единой таблицы приходов
"""

sql_query = (f"SELECT * FROM  'incmcattb' ")
income = pd.read_sql(sql_query, con)
income_category = dict(zip(income.id, income.cname))

In [6]:
income_category

{2: 'traineeship',
 3: 'excursion',
 6: 'salary',
 7: 'advance',
 8: 'salary',
 9: 'grands',
 10: 'different resources',
 11: 'transfer'}

In [7]:
sql_query = (f"SELECT * FROM  xpncetb ")
expence = pd.read_sql(sql_query, con)
expence

,id,cid,desc,itmdate,amnt,time,img,paymthrd,withwhome,voice,location
0,1,6,,2014-07-23,300,23:53,,0,None,None,None
1,2,11,,2014-07-23,200,23:55,,0,None,None,None
2,3,3,Пущино - Серпухов,2014-07-24,62,14:44,,0,None,None,None
3,4,7,шампунь\nгель для душа\nдве пасты\nдве щетки,2014-07-24,660,16:4,,0,None,None,None
4,5,2,Pazzo,2014-07-24,1000,17:14,,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
2912,2916,6,крупы,2020-03-11,530,16:56,,0,None,None,None
2913,2917,3,москва,2020-03-16,590,16:56,,0,None,None,None
2914,2918,3,москва,2020-03-13,590,16:56,,0,None,None,None
2915,2919,3,москва,2020-03-02,240,16:57,,0,None,None,None


In [8]:
sql_query = (f"SELECT * FROM  'incmetb' ")
income = pd.read_sql(sql_query, con)
income

,id,cid,desc,itmdate,amnt,time,img,paymthrd,withwhome,voice,location
0,1,3,Excursion from “Pansionat“,2014-07-15,3000,14:55,,0,None,None,None
1,2,7,,2014-07-24,3000,17:28,,0,None,None,None
2,3,3,,2014-07-25,1000,14:35,,0,None,None,None
3,4,10,форсайт,2014-08-03,500,12:11,,0,None,None,None
4,5,9,Оклад,2014-08-06,7294,18:26,,1,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
306,308,8,,2019-12-31,14424,15:20,,0,None,None,None
307,309,7,,2020-01-04,6000,12:44,,0,None,None,None
308,310,8,,2020-01-27,32187,12:44,,0,None,None,None
309,311,7,,2020-02-17,6000,11:9,,0,None,None,None


In [9]:
expence = expence[['cid', 'desc', 'itmdate', 'amnt', 'time']]

In [10]:
expence['category'] = None

/home/andr/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
for idx, row in expence.iterrows():
    expence.loc[idx, 'category'] = expence_category[row.cid]

/home/andr/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [12]:
expence

,cid,desc,itmdate,amnt,time,category
0,6,,2014-07-23,300,23:53,food
1,11,,2014-07-23,200,23:55,phone
2,3,Пущино - Серпухов,2014-07-24,62,14:44,travel
3,7,шампунь\nгель для душа\nдве пасты\nдве щетки,2014-07-24,660,16:4,shopping
4,2,Pazzo,2014-07-24,1000,17:14,entertainment
...,...,...,...,...,...,...
2912,6,крупы,2020-03-11,530,16:56,food
2913,3,москва,2020-03-16,590,16:56,travel
2914,3,москва,2020-03-13,590,16:56,travel
2915,3,москва,2020-03-02,240,16:57,travel


In [13]:
income = income[['cid', 'desc', 'itmdate', 'amnt', 'time']]

In [14]:
income['category'] = None

/home/andr/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
for idx, row in income.iterrows():
    income.loc[idx, 'category'] = income_category[row.cid]

In [16]:
income

,cid,desc,itmdate,amnt,time,category
0,3,Excursion from “Pansionat“,2014-07-15,3000,14:55,excursion
1,7,,2014-07-24,3000,17:28,advance
2,3,,2014-07-25,1000,14:35,excursion
3,10,форсайт,2014-08-03,500,12:11,different resources
4,9,Оклад,2014-08-06,7294,18:26,grands
...,...,...,...,...,...,...
306,8,,2019-12-31,14424,15:20,salary
307,7,,2020-01-04,6000,12:44,advance
308,8,,2020-01-27,32187,12:44,salary
309,7,,2020-02-17,6000,11:9,advance


In [17]:
expence['tab_label'] = 'expence'
income['tab_label'] = 'income'

/home/andr/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/andr/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
full_table = pd.concat([income, expence])

In [22]:
full_table['timedate'] = full_table['itmdate'] + ' ' + full_table['time']

In [24]:
full_table['Time_series'] = pd.to_datetime(full_table['timedate'], format='%Y-%m-%d %H:%M', errors='ignore')
full_table = full_table.set_index('Time_series')

In [26]:
full_table = full_table.sort_index()

In [27]:
full_table

,cid,desc,itmdate,amnt,time,category,tab_label,timedate
Time_series,,,,,,,,
2014-05-30 20:27:00,16,велик + замок,2014-05-30,16150,20:27,sport,expence,2014-05-30 20:27
2014-07-15 14:55:00,3,Excursion from “Pansionat“,2014-07-15,3000,14:55,excursion,income,2014-07-15 14:55
2014-07-23 23:53:00,6,,2014-07-23,300,23:53,food,expence,2014-07-23 23:53
2014-07-23 23:55:00,11,,2014-07-23,200,23:55,phone,expence,2014-07-23 23:55
2014-07-24 14:44:00,3,Пущино - Серпухов,2014-07-24,62,14:44,travel,expence,2014-07-24 14:44
...,...,...,...,...,...,...,...,...
2020-03-11 16:56:00,6,крупы,2020-03-11,530,16:56,food,expence,2020-03-11 16:56
2020-03-13 16:56:00,3,москва,2020-03-13,590,16:56,travel,expence,2020-03-13 16:56
2020-03-14 16:42:00,6,бананы и тп,2020-03-14,89,16:42,food,expence,2020-03-14 16:42


In [28]:
# Подгрузка файла в формате которого нужно сохранить данные
pattern_table = pd.read_csv('expensemanager.csv')

In [33]:
pattern_columns = pattern_table.columns

In [35]:
final_table = pd.DataFrame(columns=pattern_columns)

In [37]:
pattern_table

,Date,Amount,Category,Subcategory,Payment Method,Description,Ref/Check No,Payee/Payer,Status,Receipt Picture,Account,Tag,Tax,Quantity,Split Total,Row Id,Type Id
0,2020-08-27,-123.00,Home Office,Computer,Cash,NaN,NaN,Walmart,Cleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,1,NaN
1,2020-09-07,-617.05,Food,Snack,DebitCard,NaN,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,2,NaN
2,2020-09-07,-5.00,Food,Snack,DebitCard,Описание с запятыми на русском языке,NaN,Пятёрочка,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,3,NaN
3,2020-09-07,-10.00,Health Care,Medical,DebitCard,список: а б с.,NaN,Пятёрочка,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,4,NaN
4,2020-09-07,20.00,Income,Salary,DebitCard,NaN,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,5,NaN


In [36]:
final_table

,Date,Amount,Category,Subcategory,Payment Method,Description,Ref/Check No,Payee/Payer,Status,Receipt Picture,Account,Tag,Tax,Quantity,Split Total,Row Id,Type Id


In [44]:
count = 0
for idx, row in full_table.iterrows():
    if row['tab_label'] == 'expence':
        value = -1 * float(row['amnt'])
    else:
        value = float(row['amnt'])
        
    final_table.loc[count] = [
        row['itmdate'],
        value,
        row['category'],
        None,
        'Cash',
        row['desc'],
        np.NaN,
        np.NaN,
        'Uncleared',
        np.NaN,
        'Personal Expense',
        np.NaN,
        np.NaN,
        '1 PCS',
        np.NaN,
        count + 1,
        np.NaN
    ]
    
    count += 1

In [45]:
final_table

,Date,Amount,Category,Subcategory,Payment Method,Description,Ref/Check No,Payee/Payer,Status,Receipt Picture,Account,Tag,Tax,Quantity,Split Total,Row Id,Type Id
0,2014-05-30,-16150,sport,None,Cash,велик + замок,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,1,NaN
1,2014-07-15,3000,excursion,None,Cash,Excursion from “Pansionat“,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,2,NaN
2,2014-07-23,-300,food,None,Cash,,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,3,NaN
3,2014-07-23,-200,phone,None,Cash,,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,4,NaN
4,2014-07-24,-62,travel,None,Cash,Пущино - Серпухов,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223,2020-03-11,-530,food,None,Cash,крупы,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,3224,NaN
3224,2020-03-13,-590,travel,None,Cash,москва,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,3225,NaN
3225,2020-03-14,-89,food,None,Cash,бананы и тп,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,3226,NaN
3226,2020-03-15,-172,different expenses,None,Cash,,NaN,NaN,Uncleared,NaN,Personal Expense,NaN,NaN,1 PCS,NaN,3227,NaN


In [46]:
final_table.to_csv('final_exp_manager.csv', sep=';')

Что нужно сделать:

+ переопределить категории и подкатегории
+ сохранить в формате, принятом в новом Expence Manager